## 如何将训练好的word2vec模型与后续的NLP任务有机结合起来。它们是：
- **词汇相似度度量**
- **文本分类**
- **词汇分类**

这里会用到NLP领域两个重要的库 ---gensim和keras，笔者将gensim模型的封装接口融入到keras的处理流程中，将二者无缝对接，文本挖掘的操作流程将会大大顺畅！

这里，笔者将会用到的一个重要的封装器是gensim.models.keyedvectors中的方法get_keras_embedding。

**建议使用环境tensorflow1.4,keras2.2.5**
**确保tensorflow版本为1系列，并与keras版本相对应，否则会出现不匹配error**


In [1]:
#导入要使用到的库
import jieba
from gensim.models import word2vec

In [2]:
# 创建一个Demo语句集用以训练一个word2vec模型。
sentences = [    
    ['social_listening', '市场分析', '调研'],  
    ['文本挖掘', 'NLP', '文本处理', '情感分析', 'social_listening', '时效性'],   
    ['口碑监测', '声量', '情感分析', '语义分析'],   
    ['声量', '口碑监测', '时效性', '调研'],  
    ['典型意见挖掘', '文本挖掘', '语义分析'],   
    ['social_listening','NLP'],    
    ['文本挖掘', 'social_listening'],    
    ['市场分析', '用户画像', '口碑监测'],    
    ['social_listening', '用户画像', '声量']
]
sentences

[['social_listening', '市场分析', '调研'],
 ['文本挖掘', 'NLP', '文本处理', '情感分析', 'social_listening', '时效性'],
 ['口碑监测', '声量', '情感分析', '语义分析'],
 ['声量', '口碑监测', '时效性', '调研'],
 ['典型意见挖掘', '文本挖掘', '语义分析'],
 ['social_listening', 'NLP'],
 ['文本挖掘', 'social_listening'],
 ['市场分析', '用户画像', '口碑监测'],
 ['social_listening', '用户画像', '声量']]

In [3]:
# 传入适当的参数来创建word2vec模型。
model=word2vec.Word2Vec(sentences,size=100,min_count=1,hs=1)

## 1、与Keras进行融合 --- 词汇相似度度量
### 通过计算词汇向量的余弦距离来度量两个词汇之间的相似度。

In [4]:
import numpy as np
from keras.engine import Input
from keras.models import Model
from keras.layers.merge import dot

Using TensorFlow backend.
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [5]:
# 使用get_keras_embedding方法返回可用的层
wv=model.wv
embedding_layer=wv.get_keras_embedding()

In [6]:
# 构建Keras模型
input_a=Input(shape=(1,),dtype='int32',name='input_a')
input_b=Input(shape=(1,),dtype='int32',name='input_b')
embedding_a=embedding_layer(input_a)
embedding_b=embedding_layer(input_b)
similarity=dot([embedding_a,embedding_b],axes=2,normalize=True)
keras_model=Model(inputs=[input_a,input_b],outputs=similarity)
keras_model.compile(optimizer='sgd',loss='mse')

In [7]:
word_a = '文本挖掘'
word_b = 'social_listening'# 输出是两个词之间的余弦距离（一种文本相似性度量）
output = keras_model.predict([np.asarray([model.wv.vocab[word_a].index]), np.asarray([model.wv.vocab[word_b].index])])
print (output[0][0][0])

0.108582124


In [8]:
word_a = '文本挖掘'
word_b = '市场分析'


# 输出是两个词之间的余弦距离（一种文本相似性度量）
output = keras_model.predict([np.asarray([model.wv.vocab[word_a].index]), np.asarray([model.wv.vocab[word_b].index])])


print (output[0][0][0])

-0.06890371


In [9]:
word_a = '用户画像'
word_b = 'social_listening'
# 输出是两个词之间的余弦距离（一种文本相似性度量）
output = keras_model.predict([np.asarray([model.wv.vocab[word_a].index]), np.asarray([model.wv.vocab[word_b].index])])
# print (output[0][0][0])
output

array([[[-0.03713847]]], dtype=float32)

# 2、与Keras进行融合 --- 文本分类
进一步了解在处理有监督（分类）任务时gensim的Word2Vec模型如何与Keras集成，笔者以一个demo文本分类数据集（包含教育、娱乐、科技和体育这4个类别）。 

In [10]:
import os
import sys
import keras
import numpy as np
from gensim.models import word2vec
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Input,Dense,Flatten
from keras.layers import Conv1D,MaxPooling1D
import pandas as pd

In [11]:
# 加载数据集，因为是做示例用，所以仅抽取少部分数据。
data=pd.read_excel(r'datasets/toutiaodata.xlsx').sample(frac=0.05)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/toutiaodata.xlsx'